In [42]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

In [43]:
manual_validation = pd.read_csv(os.path.join(base_dir,'data','02_ground_truth_data','manual_validation','qanda_validated_2.csv'), dtype=str)
manual_validation['stance'] = manual_validation.label.astype(int)
manual_validation['uid'] = manual_validation.UID

In [44]:
group_manual_validation = pd.read_csv(os.path.join(base_dir,'data','02_ground_truth_data','group_qanda_manual_annotation.csv'), dtype=str)
group_manual_validation['stance'] = group_manual_validation.label.astype(int)
group_manual_validation['uid'] = group_manual_validation.UID

In [47]:
df = pd.merge(group_manual_validation[['uid','stance','Person']], manual_validation[['uid','stance']], on='uid', how='left')

In [48]:
from sklearn.metrics import cohen_kappa_score

In [49]:
cohen_kappa_score(df.stance_x, df.stance_y)

0.5146520146520146

In [53]:
cohen_kappa_score(df[~df.Person.isin(['4','7','8'])].stance_x, df[~df.Person.isin(['4','7','8'])].stance_y)

0.7047368421052631

In [50]:
df.groupby('Person').apply(lambda d: cohen_kappa_score(d.stance_x, d.stance_y))

Person
1    0.641791
3    0.693878
4    0.239437
5    0.669118
6    0.794937
7    0.283186
8    0.231499
dtype: float64

In [56]:
df.groupby('stance_x').apply(lambda d: cohen_kappa_score(d.stance_y,d.stance_x))

stance_y
-1    0.0
 1    0.0
 2    0.0
dtype: float64

In [22]:
mask = (df.stance_x == -1) | (df.stance_y == -1)

In [17]:
cohen_kappa_score(df.stance_x, df.stance_y)

0.5146520146520146

In [23]:
cohen_kappa_score(df[mask].stance_x, df[mask].stance_y)

-0.29441624365482233

In [30]:
group_manual_validation[group_manual_validation.stance != -1]

,UID,username,name,protected?,url,label,left,right,far-right,N/A,Notes,Person,stance,uid
0,1271723012409049088,Bonzo08367039,Bonzo,FALSE,twitter.com/Bonzo08367039,1,1,NaN,NaN,NaN,Anti-Abbott,1,1,1271723012409049088
1,1275613388991651846,Podwhale2,Podwhale,FALSE,twitter.com/Podwhale2,1,1,NaN,NaN,NaN,Anti-Morrison,1,1,1275613388991651846
2,55978337,Andrew8754,Andrew,FALSE,twitter.com/Andrew8754,1,1,NaN,NaN,NaN,Anti-Morrison,1,1,55978337
3,185932331,AlanTudgeMP,Alan Tudge,FALSE,twitter.com/AlanTudgeMP,2,NaN,1,NaN,NaN,Liberal Party,1,2,185932331
4,1046581409861783553,itslilzap,zap,FALSE,twitter.com/itslilzap,2,NaN,1,NaN,NaN,Against Pfizr,1,2,1046581409861783553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,1237931074631630850,UNKNOWN56236823,UNKNOWN,FALSE,twitter.com/UNKNOWN56236823,2,NaN,NaN,1,NaN,Retweets conspiracy theories,8,2,1237931074631630850
175,30416880,katek3,Kate Kneebone,FALSE,twitter.com/katek3,1,1,NaN,NaN,NaN,Some left leaning content,8,1,30416880
177,171437375,jkalbrechtsen,janet albrechtsen,FALSE,twitter.com/jkalbrechtsen,2,NaN,1,NaN,NaN,Right leaning journalist,8,2,171437375
178,63942152,JoshFrydenberg,Josh Frydenberg,FALSE,twitter.com/JoshFrydenberg,2,NaN,1,NaN,NaN,Liberal minister,8,2,63942152
